#### ==============================================================
#### Instacart Project – Phase 2: Clean & Transform
#### - Load raw CSVs
#### - Concatenate prior + train order_products
#### - Filter out eval_set="test" from orders
#### - Save clean CSVs to data_clean
#### ==============================================================

In [ ]:
import pandas as pd
import os

pd.set_option("display.max_columns", None)

#### --------------------------------------------------------------
#### Load raw data
#### --------------------------------------------------------------

In [33]:
DATA_PATH = r'C:\Users\User\Documents\Projects\instacart_project\data_raw'
DATA_CLEAN_PATH = r'C:\Users\User\Documents\Projects\instacart_project\data_clean'

os.listdir(DATA_PATH)

files = {
    "aisles": "aisles.csv",
    "departments": "departments.csv",
    "orders": "orders.csv",
    "order_products_prior": "order_products__prior.csv",
    "order_products_train": "order_products__train.csv",
    "products": "products.csv",
}

dfs = {}

for name, fname in files.items():
    path = os.path.join(DATA_PATH, fname)
    df = pd.read_csv(path)
    dfs[name] = df
    print(f"{name}: {df.shape[0]:,} rows × {df.shape[1]} columns")


aisles: 134 rows × 2 columns
departments: 21 rows × 2 columns
orders: 3,421,083 rows × 7 columns
order_products_prior: 32,434,489 rows × 4 columns
order_products_train: 1,384,617 rows × 4 columns
products: 49,688 rows × 4 columns


In [ ]:
## --------------------------------------------------------------
## Concatenate PRIOR + TRAIN into a single order_products table
## --------------------------------------------------------------

In [35]:
order_products_prior = dfs["order_products_prior"]
order_products_train = dfs["order_products_train"]

print("\norder_products_prior.info():")
print(order_products_prior.info())

order_products = pd.concat(
    [order_products_prior, order_products_train],
    axis=0,
    ignore_index=True
)

print("\nCombined order_products shape:", order_products.shape)

# Save combined order_products to data_clean
order_products.to_csv(
    os.path.join(DATA_CLEAN_PATH, "order_products.csv"),
    index=False
)



order_products_prior.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB
None

Combined order_products shape: (33819106, 4)


## --------------------------------------------------------------
## Clean orders table: remove eval_set = 'test'
## --------------------------------------------------------------

In [36]:
orders = dfs["orders"]

orders_rmv = orders[orders["eval_set"] != "test"]
print("\norders_rmv.info():")
print(orders_rmv.info())

orders_rmv.to_csv(
    os.path.join(DATA_CLEAN_PATH, "orders.csv"),
    index=False
)


orders_rmv.info():
<class 'pandas.core.frame.DataFrame'>
Index: 3346083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 204.2+ MB
None


In [ ]:
## --------------------------------------------------------------
## quick join test in pandas (actual joins in PostgreSQL)
## --------------------------------------------------------------

In [39]:
orders_clean = orders_rmv.merge(order_products,on="order_id",how="left")

print("\norders_clean shape (for sanity check):", orders_clean.shape)


orders_clean shape (for sanity check): (33819106, 10)
